In [1]:
%matplotlib widget

# Verify and demonstrate the M-k relation

In [2]:
from bmcs_cross_section.mkappa import MKappa
from bmcs_utils.api import InteractiveWindow

mc = MKappa(low_kappa=-0.000014, high_kappa=0.00003)
bd = mc.beam_design
css = bd.cross_section_shape
csl = bd.cross_section_layout
csl.reinforcement.z_j = [10]
InteractiveWindow([mc, bd, mc.matrix, css, csl]).interact()

Float slider 0 -1.4e-05 3e-05 4.3999999999999997e-07
Solve for eps_bot_t


In [3]:
mc.get_kappa_M(0.5)

1.12910051893192e-07

In [ ]:
import numpy as np
import matplotlib.pylab as plt
xdata = np.array([0,1,0.9, 0.8, 2])
ydata = np.array([0,1,2,3, 4])

In [ ]:
plt.plot(xdata, ydata)
plt.show()

In [ ]:
diffs = xdata[:,np.newaxis] - xdata[np.newaxis,:]
diffs

In [ ]:
ij = np.mgrid[0:5:1,0:5:1]
ij

In [ ]:
diffs[np.where(ij[1] >= ij[0])] = 0

In [ ]:
diffs

In [13]:
i_x = np.argmin(diffs, axis=1)

In [14]:
xdata[i_x]

array([0., 1., 1., 1., 2.])

In [15]:
plt.plot(xdata[i_x], ydata)
plt.show()